# Floor Plan Optimization

## Context
The floorplan optimization seeks to find an optimal organization of machines on a factory floor to minimize the total travel time to produce a portfolio of parts. For each part in a catalog, we have:
* A volume of parts produced
* An ordered list of the machines operating on the materials to produce the part

The floorplan is shown below. We assume each machine has unit length/width and is only accessible on one side. Machines are accessed by an aisle, with 3 intersecting aisles at the top, middle, and bottom of the floorplan as seen in the image below.

<img src="Layout.png" alt="drawing" width="200"/>

## To Do:
* Add distances to mid and bottom, with shortest distance
* Speed up loop

In [ ]:
import pandas as pd
import numpy as np
import random
from itertools import tee
import itertools
from dfply import *
import matplotlib.pyplot as plt

# Asset Definition
In this example, we will have 50 machines producing 3000 parts.

First a list of machines is generated.

In [ ]:
# Machine List
machineList = ['M' + str(x) for x in range(0,50)]
print('Machines:')
print(machineList)

Next, we generate a list of parts and a build volume for each part.

In [ ]:
# Part Table
## Each part has a key, volume, and list of machines
partTable = pd.DataFrame.from_dict({'PN': list(range(0,3000)),
                                    'Volume': np.random.exponential(5000, 3000)})

partTable.PN = partTable.PN.astype('O')

print('Part Table:')
print(partTable.head())

In addition, each part gets a sequence of machine operations pulled from the machine list. The data is formated in melted format, i.e. each row describes the trip between machines.

In [ ]:
# For each part, generate a list of machines to be visited

# Schema:
# pn  |  tripstart   | tripend
# 0   |  M1          | M2
# 0   |  M2          | M5


# Function to create pairwise list of machines
def pairwise(sequence):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iter(sequence))
    next(b, None)
    return zip(a, b)

# Provision an empty dataframe for results
partOperations = pd.DataFrame(columns=['PN', 'tripStartMachine', 'tripEndMachine'])

# For each part...
for PN in list(range(0,3000)):
    # Generate a sequence of operations (machines)
    operations = random.sample(machineList, np.random.randint(low=2, high=15))
    operations.insert(0, 'start')
    # Pariwise dataframe
    operations = pd.DataFrame(list(pairwise(operations)), columns=['tripStartMachine', 'tripEndMachine'])
    operations['PN'] = PN
    # Append to results dataframe
    partOperations = partOperations.append(operations, sort=False)

partOperations.head()

Finally, we prescribe a floorplan with slots for at least as many machines as we have listed. Here we also measure distances between all slots. This calculation is done once so that we don't recalculate distances between machines on the fly.

In [ ]:
# Floorplan

## Set parameters of layout
aisleCount = 10
machinesPerSide = 5

## Generate list of distances between each slot and each aisle
## Schema:
## slotID | aisle | position | dTop | dMid | dBot

## Generate coordinates
slotDistances = pd.DataFrame(list(itertools.product(list(range(aisleCount)), list(range((2*machinesPerSide) + 3)))),
                             columns=['aisle', 'position'])

## Measure distances
slotDistances['dTop'] = ((2*machinesPerSide) + 3 - 1) - slotDistances[['position']]
slotDistances['dBot'] = slotDistances[['position']]
slotDistances['dMid'] = np.abs(slotDistances[['position']] - (machinesPerSide + 1))

## Remove aisles
slotDistances = slotDistances[~slotDistances.position.isin([0, machinesPerSide+1, 2*machinesPerSide+2])]

## Add slot ID
slotDistances['slotID'] = range(0, len(slotDistances))
slotDistances.set_index('slotID', inplace=True)

slotIndex = slotDistances.index

# Pairwise slot distances
pairDistances = []
for x in list(itertools.product(slotDistances.index, slotDistances.index)):
    dTopA = slotDistances.dTop.loc[x[0]]
    dMidA = slotDistances.dMid.loc[x[0]]
    dBotA = slotDistances.dBot.loc[x[0]]
    aisleA = slotDistances.aisle.loc[x[0]]
    dTopB = slotDistances.dTop.loc[x[1]]
    dMidB = slotDistances.dMid.loc[x[1]]
    dBotB = slotDistances.dBot.loc[x[1]]
    aisleB = slotDistances.aisle.loc[x[1]]
    
    if aisleA == aisleB:
        dist = np.abs(dTopA - dTopB)
    else:
        dist = min((dTopA + dTopB), (dMidA + dMidB), (dBotA + dBotB)) + np.abs(aisleA - aisleB)
    
    pairDistances.append((x[0], x[1], dist))
    
# Convert to dataframe
pairDistances = pd.DataFrame(pairDistances, columns=['tripStartSlotID', 'tripEndSlotID', 'distance'])

# Add distances from start to each slot
startDistances = slotDistances[['dMid', 'aisle']].reset_index()
startDistances.columns = ['tripEndSlotID', 'dMid', 'aisle']
startDistances['distance'] = startDistances.dMid + startDistances.aisle + 1
startDistances['tripStartSlotID'] = 'start'
pairDistances = pairDistances.append(startDistances[['tripStartSlotID', 'tripEndSlotID', 'distance']], sort=False)

# Clean up typing
pairDistances['tripEndSlotID'] = pairDistances.tripEndSlotID.astype('O')

In [ ]:
slotIndex

In [ ]:
slotDistances.head()

In [ ]:
pairDistances.tail()

# Assignment and Fitness

In [ ]:
# Sample len(machine) slots from slot index to allocate machines to slots
slotAssignment = pd.DataFrame({'machineID': machineList, 
                               'slotID': random.sample(list(slotIndex), len(machineList))})

# Join a complete list of slotIDs to include blanks
slotAssignment = slotAssignment.merge(pd.DataFrame({'slotID': list(slotIndex)}), how='right').sort_values('slotID')

# Add start position
slotAssignment = slotAssignment.append(pd.DataFrame({'machineID': ['start'], 'slotID': ['start']}), sort=False)
slotAssignment = slotAssignment.reset_index(drop=True)

slotAssignment.head()

In [ ]:
# Define the fitness function to measure quality of a solution
def costFunction(floorplan, partOps, pairDists, partVolumes):
    # Map machines in part ops to slots in floorplan
    fitnessCalc = partOps.merge(floorplan, how='left', left_on='tripStartMachine', right_on='machineID')
    fitnessCalc = fitnessCalc.merge(floorplan, how='left', left_on='tripEndMachine', right_on='machineID')
    ## Clean up the join
    fitnessCalc = fitnessCalc[['PN', 'tripStartMachine', 'tripEndMachine',
                              'slotID_x', 'slotID_y']]
    fitnessCalc.columns = ['PN', 'tripStartMachine', 'tripEndMachine',
                           'tripStartSlotID', 'tripEndSlotID']
    
    # Add distances between slot pairs
    fitnessCalc = fitnessCalc.merge(pairDists, how='left',
                                   on=['tripStartSlotID', 'tripEndSlotID'])
    
    # Part volume
    fitnessCalc = fitnessCalc.merge(partVolumes, how='left', on='PN')

    # calculate total travels
    fitnessCalc['weightedDistance'] = fitnessCalc.distance * fitnessCalc.Volume
    result = sum(fitnessCalc.weightedDistance)
        
    return(result)

# Test the fitness function on the initial configuration
initCost = costFunction(slotAssignment, partOperations, pairDistances, partTable)
initCost

In [ ]:
# Define function to swap n pairs of slots during annealing
def iteratePlan(floorplan, slots, n=2):
    while n > 0:
        # Take 2 slots at random
        slotsToSwap = random.sample(list(slots), 2)
        subdat = floorplan.loc[floorplan.slotID.isin(slotsToSwap), :]
        # Reverse order
        subdat.loc[:,'machineID'] = subdat.loc[:,'machineID'].values[::-1]
        # Replace in floorplan
        floorplan = floorplan.loc[~floorplan.slotID.isin(slotsToSwap), :]
        floorplan = floorplan.append(subdat)
        # Increment down
        n = n-1
    return(floorplan)

# See a sample swap
iteratePlan(slotAssignment, slotIndex).head()

## Annealing

In [ ]:
# Initialize parameters
def acceptanceProbability(oldCost, newCost, temp):
    return np.exp((oldCost - newCost) / temp)

def anneal(sol, partOps, pairDist, partTbl, slotIdx):
    # Initialize the annealer
    old_cost = costFunction(sol, partOps, pairDist, partTbl)
    T = 1.0
    Tmin = 0.0001
    alpha = 0.85
    Tsamples = 20
    Tlist = []
    Clist = []
    
    # Start main loop
    iloop = 1
    while T > Tmin:
        i = 1
        while i <= Tsamples:
            new_sol = iteratePlan(sol, slotIdx, n=1)
            new_cost = costFunction(new_sol, partOps, pairDist, partTbl)
            ap = acceptanceProbability(old_cost, new_cost, T)
            if ap > random.random():
                sol = new_sol
                old_cost = new_cost
            i += 1
        print("Temperature: {}    | Score: {}".format(T, old_cost))
        Tlist = Tlist + [iloop]
        Clist = Clist + [old_cost]
        T = T*alpha
        iloop += 1
    return sol, old_cost, Tlist, Clist

    
finalLayout, cost, Tlist, Clist= anneal(slotAssignment, partOperations, pairDistances, partTable, slotIndex)

In [ ]:
results = pd.DataFrame({'temperature': Tlist, 'cost': Clist})

In [ ]:
plt.figure(figsize=(6,8))
plt.scatter(results.temperature, results.cost)

In [ ]:
print("Percent improvement over random: {}%".format(round(100 * (initCost - cost) / initCost,2)))

In [ ]:
finalLayout = finalLayout[~(finalLayout.slotID == 'start')]
finalLayout.slotID = finalLayout.slotID.astype('int')

In [ ]:
fillPlot = slotDistances.reset_index().merge(finalLayout, how='left')
fillPlot.machineID[fillPlot.machineID.notna()] = 1
fillPlot.machineID[fillPlot.machineID.isna()] = 0
fillPlot.machineID = fillPlot.machineID.astype(float)
fillPlot.head(10)
fillPlot.dtypes
plt.scatter(fillPlot.aisle, fillPlot.position, c = fillPlot.machineID)